In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy
from matplotlib import cm

from graspologic.embed import AdjacencySpectralEmbed, select_dimension
from graspologic.align import SeedlessProcrustes, SignFlips
from graspologic.plot import heatmap
from hyppo.ksample import KSample
from tqdm import tqdm

from utils import load_data

%matplotlib inline

In [2]:
A_l, nodes_l, A_r, nodes_r, hemispheres = load_data()

In [3]:
test_args = dict(
    reps = 5000,
    workers = -1,
    auto = False,
)

pvalues = []

for n_components in tqdm(range(1, 6)):
    ase = AdjacencySpectralEmbed(n_components=n_components, algorithm='full', diag_aug=True, check_lcc=False)
    X1_hat = ase.fit_transform(A_l)
    X2_hat = ase.fit_transform(A_r)

    X1_opt = SeedlessProcrustes().fit_transform(X1_hat, X2_hat)
    X1_median = SignFlips(criterion="median").fit_transform(X1_hat, X2_hat)
    
    dcorr_median = KSample("Dcorr").test(X1_median, X2_hat, **test_args)
    dcorr_opt = KSample("Dcorr").test(X1_opt, X2_hat, **test_args)
    
    mgc_median = KSample("MGC").test(X1_median, X2_hat, **test_args)
    mgc_opt = KSample("MGC").test(X1_opt, X2_hat, **test_args)
    
    pvalues.append([n_components, mgc_opt[1], dcorr_opt[1], mgc_median[1], dcorr_median[1]])

100%|█████████████████████████████████████████████████████| 5/5 [04:05<00:00, 49.13s/it]


In [4]:
df = pd.DataFrame(pvalues, columns = ['n_components', 'MGC+OPT', 'DCorr+OPT', 'MGC+Median', 'DCorr+Median'])

In [5]:
print(df.T.to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &       0 &       1 &       2 &       3 &       4 \\
\midrule
n\_components &  1.0000 &  2.0000 &  3.0000 &  4.0000 &  5.0000 \\
MGC+OPT      &  0.9900 &  1.0000 &  0.9976 &  0.9992 &  0.9534 \\
DCorr+OPT    &  0.9864 &  1.0000 &  0.9970 &  0.9982 &  0.9600 \\
MGC+Median   &  0.9888 &  1.0000 &  0.0002 &  0.0002 &  0.0002 \\
DCorr+Median &  0.9894 &  0.9996 &  0.0044 &  0.0090 &  0.0256 \\
\bottomrule
\end{tabular}



In [6]:
df.to_csv("results/table1.csv", index=False)